# Filter Analysis

This notebook allows you to gain familiarity with multi-stage filters.

The slider bars control the cutoff frequencies for the highpass filter (HPF) and the lowpass filter (LPF), which are cascaded to form bandpass filters.

There is also a slider bar for the filter order ($N$). This number indicates how many identical HPF and LPF stages we want to use in a row to get a sharper filter response.
For this example, the total filter order is $2N$:  there are $N$ identical lowpass filters, followed by $N$ identical highpass filters, all separated by buffers to prevent loading effects.


In the time-domain plots:
 - red is the 60Hz noise
 - green is the 600Hz desired signal
 - blue is the 60kHz noise
 - black is the sum of all the voltages
 
 
 If there were no noise present, the black signal (total voltage) would overlap the green signal (desired signal) exactly.
 However, because noise is present, the black signal deviates from the green, as can be seen for the input voltage waveform.

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex
from ipywidgets import interactive, widgets, Layout
from scipy import signal

In [ ]:
# setup signals
w_60 = 2 * np.pi * 60      # noise frequency (in rad/sec)
w_600 = 2 * np.pi * 600    # desired signal frequency (in rad/sec)
w_60k = 2 * np.pi * 60e3   # noise frequency (in rad/sec)

in_60_A = 20e-3            # noise amplitude
in_600_A = 1e-3            # signal amplitude
in_60k_A = 10e-3           # noise amplitude

t = np.arange(0, 2 * 2 * np.pi / w_60, 1 / 10 * 2 * np.pi / w_60k)
ws = [w_60, w_600, w_60k]

##########
# Helper function
##########
def bode_data(afilter):
    my_wbode, my_magbode, my_phasebode = signal.bode(afilter, np.logspace(1, 6))
    return my_wbode, my_magbode, my_phasebode

def print_freq(w_lp, w_hp):
    print(f'f_lowpass = {np.abs(w_lp) / (2 * np.pi) / 1000:0.3f} kHz')
    print(f'f_highpass = {np.abs(w_hp)/(2 * np.pi) / 1000:0.3f} kHz')

##########
# Sliders
##########
# Slider limits
lp_min = 2 * np.pi
lp_max = 2 * np.pi * 1e5
hp_min = 2 * np.pi
hp_max = 2 * np.pi * 1e5
lp_init = 2 * np.pi * 6000
hp_init = 2 * np.pi * 189

w_lp_slider = widgets.FloatLogSlider(value=lp_init, min=np.log10(lp_min), max=np.log10(lp_max), step=.05, description='$\omega_\mathrm{lowpass}$', layout=Layout(width='100%'))
w_hp_slider = widgets.FloatLogSlider(value=hp_init, min=np.log10(hp_min), max=np.log10(hp_max), step=.05, description='$\omega_\mathrm{highpass}$', layout=Layout(width='100%'))
N_slider = widgets.IntSlider(value=1, min=1, max=6, description='Filter order ($N$)', style={'description_width': 'initial'}, layout=Layout(width='50%'))

##########
# Initial plots
##########
# Initial filter
filter = signal.ZerosPolesGain([0], 
                               [-lp_init, -hp_init], 
                               np.power(lp_init, 1))

# Bode plots
wbode1, magbode1, phasebode1 = bode_data(filter)

fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

_mag, = ax1.semilogx(wbode1 / (2 * np.pi), magbode1)
ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel('Magnitude [dB]')
ax1.set_ylim([-100, 0.5])
ax1.grid()
_phase, = ax2.semilogx(wbode1 / (2 * np.pi), phasebode1)
ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('Phase [deg]')
ax2.grid()
fig1.tight_layout()

#################
# Filter response
#################
# Input plot
_, Hs = signal.freqresp(filter, ws)
in_60 = in_60_A * np.cos(w_60 * t)
in_600 = in_600_A * np.cos(w_600 * t)
in_60k = in_60k_A * np.cos(w_60k * t)
in_total = in_60 + in_600 + in_60k

fig2, (ax3, ax4) = plt.subplots(1, 2, figsize=(10,5))
ax3.plot(t, in_60, 'r', label='60Hz noise')
ax3.plot(t, in_600, 'g', label='600Hz desired')
ax3.plot(t, in_60k, 'b', alpha=0.3, label='60kHz noise')
ax3.plot(t, in_total, 'k', alpha=0.5, label='Total signal')
ax3.legend(loc="upper right")
ax3.set_xlim([t.min(), t.max()])
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Input Voltages [V]')

# Output plot
out_60 = in_60_A * np.abs(Hs[0]) * np.cos(w_60 * t + np.angle(Hs[0]))
out_600 = in_600_A * np.abs(Hs[1]) * np.cos(w_600 * t + np.angle(Hs[1]))
out_60k = in_60k_A * np.abs(Hs[2]) * np.cos(w_60k * t + np.angle(Hs[2]))
out_total = out_60 + out_600 + out_60k

# fig4, ax4 = plt.subplots()
_out0, = ax4.plot(t, out_60, 'r', label='60Hz noise')
_out1, = ax4.plot(t, out_600, 'g', label='600Hz desired')
_out2, = ax4.plot(t, out_60k, 'b', alpha=0.3, label='60kHz noise')
_out3, = ax4.plot(t, out_total, 'k', alpha=0.5, label='Total signal')
ax4.legend(loc="upper right")
ax4.set_xlim([t.min(), t.max()])
ax4.set_xlabel('Time [s]')
ax4.set_ylabel('Output Voltages [V]')
fig2.tight_layout()

########
# Update
########  
def update(w_lp_val, w_hp_val, N_val):
    # Redefine the filter
    filter = signal.ZerosPolesGain([0] * N_val, 
                                   [-w_lp_val, -w_hp_val] * N_val, 
                                   np.power(w_lp_val, N_val))
    
    # New Bode plot and display it
    wbode1, magbode1, phasebode1 = bode_data(filter)
    _mag.set_ydata(magbode1)
    _phase.set_ydata(phasebode1)
    ax2.set_ylim([phasebode1.min() - 10, phasebode1.max() + 10])
    
    print_freq(w_lp_val, w_hp_val)
    
    # Recalculate output
    _, Hs = signal.freqresp(filter, ws)
    
    out_60 = in_60_A * np.abs(Hs[0]) * np.cos(w_60 * t + np.angle(Hs[0]))
    out_600 = in_600_A * np.abs(Hs[1]) * np.cos(w_600 * t + np.angle(Hs[1]))
    out_60k = in_60k_A * np.abs(Hs[2]) * np.cos(w_60k * t + np.angle(Hs[2]))
    out_total = out_60 + out_600 + out_60k

    _out0.set_ydata(out_60)
    _out1.set_ydata(out_600)
    _out2.set_ydata(out_60k)
    _out3.set_ydata(out_total)
    ax4.set_ylim([out_total.min() - 0.01 * (out_total.max() - out_total.min()), out_total.max() + 0.01 * (out_total.max() - out_total.min())])

    fig1.canvas.draw_idle()
    fig2.canvas.draw_idle()

interactive(update, w_lp_val=w_lp_slider, w_hp_val=w_hp_slider, N_val=N_slider)
